In [16]:
import transformers
import torch
import requests
import sys

In [2]:
torch.cuda.is_available()

True

In [7]:
GUESSED = False
CORRECT_ANS = 'mbappe'
MODEL_NAME = "llama3.1:latest"

In [14]:
models = requests.get("http://localhost:11434/api/tags").json()
installed = [m["name"] for m in models["models"]]
print("Installed models:", installed)

Installed models: ['llama3:8b', 'mario:latest', 'mistral:7b', 'llama3.1:latest', 'steamdj/llama3.1-cpu-only:latest', 'mistral:latest', 'mixtral:latest', 'llama3:latest', 'snowflake-arctic-embed2:latest', 'mistral-small:latest', 'gemma3:27b', 'deepseek-r1:70b', 'deepseek-r1:32b']


In [17]:
from ollama import Client

# Initialize client (connects to the Ollama service running locally)
client = Client(host='http://localhost:11434')


def chat():
    global conversation
    print("=========WELCOME TO THIS (un)PLEASANT GUESSING GAME=========\n")
    # TODO: send image!
    print("Ask some clarification questions if you do not know what article could the image be from, or guess directly!\n")

    conversation = [
         {"role": "system", "content": """
         You are a guessing game host. The user has received an image and has to guess what Wikipedia article it comes from. They can ask you clarification questions and propose answers until they guess correctly.
         The CORRECT ANSWER (that will terminate the game) is: {CORRECT_ANS}
         The context to use for answering clarification questions: {ARTICLE_TEXT}
         """}
    ]
    while not GUESSED:
        user_input = input()
        if user_input.lower()==CORRECT_ANS:
            print("YIPPI TU AS devinE !")
            break

        # Stream response so you see it as it generates
        stream = client.chat(
            model=MODEL_NAME,
            messages=conversation + [{"role": "user", "content": user_input}],
            stream=True
        )

        reply = ""
        print("Answer: ", end="", flush=True)
        for chunk in stream:
            content = chunk["message"]["content"]
            reply += content
            sys.stdout.write(content)
            sys.stdout.flush()
        print("\n")

        # Update conversation history
        conversation.append({"role": "user", "content": user_input})
        conversation.append({"role": "assistant", "content": reply})

chat()


=========WELCOME TO THIS (un)PLEASANT GUESSING GAME=========

Ask some clarification questions if you do not know what article could the image be from, or guess directly!



 ok


Answer: You've received an image, and you need to guess what Wikipedia article it comes from. Go ahead and ask me any clarification questions you'd like (e.g., "Is the article related to a movie?", "Is it a person?", etc.) or propose an answer.

(Remember, the correct answer is: {CORRECT_ANS})

What would you like to do first?



 mbappe


YIPPI TU AS devinE !
